In [1]:
import importlib
import sys
from tqdm import tqdm
import  os
import glob
# Add the directories to the sys.path
dir_path = "C://Users//jony//Programming//Python//Anubis//anubis//" # insert your directory path
sys.path.append(dir_path + "Osiris//processing//python")
sys.path.append(dir_path + "tools")

import Analysis_tools as ATools
import proAnubis_Analysis_Tools
import Reconstruction_tools as RTools
import mplhep as hep
import Timing_tools as TTools
import rawFileReader
hep.style.use([hep.style.ATLAS])

# Specify the directory
data_list = sorted([f for f in os.listdir("data") if os.path.isfile(os.path.join("data", f))], reverse=True) ##all files in data directory sorted from the newest to the oldest

file_path = list(map(lambda p: dir_path+"//data//"+p, data_list)) # insert your file

In [ ]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(ATools)
interval = 100 # Set your monitoring chunck size
fReader = rawFileReader.fileReader(file_path[0]) # reload in the classs object
#fReader.skip_events(4000000) # Skip the first 1000 events
order = [(0,1), (1,2), (2,3), (3,4)] # Order what you want to align
max_process_event_chunk = 1000 # End the loop early
processedEvents = 0 # Initialisation

#Initialise variables to store the results
mets = [[] for tdc in range(5)]
empty_header = 0 # Counting the number of empty headers
emtpy_events_with_header = [[], []] # prepare two lists of same dimensions for line graph plotting
tdc_event_count = [[] for tdc in range(5)]# prepare for histogram plotting


with tqdm(total=max_process_event_chunk, desc="Processing Events", unit='Events') as pbar:
    while processedEvents < max_process_event_chunk:
        processedEvents += 1
        event_chunk = fReader.get_aligned_events(order=order, interval=interval) # get the aligned events
        tdc_event_count_buffer = RTools.find_tdc_event_count(event_chunk) # finding the number of headers
        [tdc_event_count[i].append(tdc_event_count_buffer[i]) for i in range(5)] # write from buffer to memory
        for idx, (i, j) in enumerate(order):
            x, y, l, m = ATools.find_tdc_alignment_metric(i, j) # determining which RPCs to use for aignment metric
            alignMet = ATools.calcAvgAlign(event_chunk, 0, x, y, l, m, i, j, processedEvents) # determine the alignment metric
            #empty_header += (alignMet == -1) # Calculating the number of empty headers
            #emtpy_events_with_header[0].append(empty_header) # write to memory
            #emtpy_events_with_header[1].append(processedEvents)
            mets[idx].append(alignMet) # write to memory
        pbar.update(1)
    